In [32]:
library(ArchR)
library(Signac)
library(Seurat)
library(ggplot2)
library(patchwork)

In [33]:
addArchRThreads(threads = 8)

Setting default number of Parallel threads to 8.



In [3]:
addArchRGenome("mm10")

Setting default genome to Mm10.



In [4]:
# 设置文件路径和样本名称
file_path <- "/mnt/disk1/xiaojk/data/mousebrain/fragments.sort.bed.gz"
sample_name <- "GAdBrainCortex_SNAREseq__fragments"

# 将文件路径和样本名称添加到相应的变量中
inputFiles <- c(file_path)
sampleNames <- c(sample_name)

# 调用createArrowFiles函数
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  sampleNames = sampleNames,
  filterTSS = 4,
  filterFrags = 1000,
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

filterFrags is no longer a valid input. Please use minFrags! Setting filterFrags value to minFrags!

filterTSS is no longer a valid input. Please use minTSS! Setting filterTSS value to minTSS!

Using GeneAnnotation set by addArchRGenome(Mm10)!

Using GeneAnnotation set by addArchRGenome(Mm10)!

ArchR logging to : ArchRLogs/ArchR-createArrows-32d9aa3acbde6f-Date-2023-09-16_Time-13-13-04.451199.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2023-09-16 13:13:04.646563 : Batch Execution w/ safelapply!, 0 mins elapsed.

(GAdBrainCortex_SNAREseq__fragments : 1 of 1) Checking if completed file exists!

2023-09-16 13:13:04.678007 : (GAdBrainCortex_SNAREseq__fragments : 1 of 1) Arrow Exists! Marking as completed since force = FALSE!, 0.001 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-32d9aa3acbde6f-Date-2023-09-16_Time-13-13-04.451199.log



In [5]:
ArrowFiles

[1] "GAdBrainCortex_SNAREseq__fragments.arrow"

In [6]:
projDLPFC1 <- ArchRProject(
  ArrowFiles = ArrowFiles,
  outputDirectory = "/mnt/disk1/xiaojk/data/mousebrain",
  copyArrows = TRUE #This is recommened so that if you modify the Arrow files you have an original copy for later usage.
)

Using GeneAnnotation set by addArchRGenome(Mm10)!

Using GeneAnnotation set by addArchRGenome(Mm10)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [7]:
projDLPFC1


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /mnt/disk1/xiaojk/data/mousebrain 
samples(1): GAdBrainCortex_SNAREseq__fragments
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 15436
medianTSS(1): 15.644
medianFrags(1): 1586

In [8]:
paste0("Memory Size = ", round(object.size(projDLPFC1) / 10^6, 3), " MB")
getAvailableMatrices(projDLPFC1)

[1] "Memory Size = 32.192 MB"

[1] "GeneScoreMatrix" "TileMatrix"

In [9]:
projDLPFC1 <- addIterativeLSI(
    ArchRProj = projDLPFC1,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI", 
    iterations = 2, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.2), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-32d9aa5d8b98ff-Date-2023-09-16_Time-13-13-08.180854.log
If there is an issue, please report to github with logFile!

2023-09-16 13:13:08.30451 : Computing Total Across All Features, 0 mins elapsed.

2023-09-16 13:13:12.420074 : Computing Top Features, 0.069 mins elapsed.

###########
2023-09-16 13:13:13.663881 : Running LSI (1 of 2) on Top Features, 0.09 mins elapsed.
###########

2023-09-16 13:13:13.774485 : Sampling Cells (N = 10000) for Estimated LSI, 0.092 mins elapsed.

2023-09-16 13:13:13.776965 : Creating Sampled Partial Matrix, 0.092 mins elapsed.

2023-09-16 13:13:19.796508 : Computing Estimated LSI (projectAll = FALSE), 0.192 mins elapsed.

2023-09-16 13:13:49.896563 : Identifying Clusters, 0.694 mins elapsed.

2023-09-16 13:14:07.65136 : Identified 3 Clusters, 0.99 mins elapsed.

2023-09-16 13:14:07.675937 : Saving LSI Iteration, 0.99 mins elapsed.

2023-09-16 13:14:32.619361 : Creating Cluster Matrix on 

In [10]:
projDLPFC1 <- addClusters(
    input = projDLPFC1,
    reducedDims = "IterativeLSI",
    method = "Seurat",
    name = "Clusters",
    resolution = 0.8
)

ArchR logging to : ArchRLogs/ArchR-addClusters-32d9aa3ccd2e0-Date-2023-09-16_Time-13-15-36.887731.log
If there is an issue, please report to github with logFile!

2023-09-16 13:15:37.077975 : Running Seurats FindClusters (Stuart et al. Cell 2019), 0.002 mins elapsed.

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 15436
Number of edges: 605018

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7267
Number of communities: 9
Elapsed time: 1 seconds


1 singletons identified. 8 final clusters.

2023-09-16 13:15:57.547199 : Testing Outlier Clusters, 0.343 mins elapsed.

2023-09-16 13:15:57.551772 : Assigning Cluster Names to 8 Clusters, 0.343 mins elapsed.

2023-09-16 13:15:57.658321 : Finished addClusters, 0.345 mins elapsed.



In [11]:
head(projDLPFC1$Clusters)
table(projDLPFC1$Clusters)

[1] "C3" "C3" "C3" "C4" "C7" "C4"


  C1   C2   C3   C4   C5   C6   C7   C8 
2237 2475 2306 5021 1492  425 1162  318 

In [12]:
projDLPFC1 <- addGroupCoverages(ArchRProj = projDLPFC1, groupBy = "Clusters")

ArchR logging to : ArchRLogs/ArchR-addGroupCoverages-32d9aa538105f8-Date-2023-09-16_Time-13-15-57.702.log
If there is an issue, please report to github with logFile!

C1 (1 of 8) : CellGroups N = 2

C2 (2 of 8) : CellGroups N = 2

C3 (3 of 8) : CellGroups N = 2

C4 (4 of 8) : CellGroups N = 2

C5 (5 of 8) : CellGroups N = 2

C6 (6 of 8) : CellGroups N = 2

C7 (7 of 8) : CellGroups N = 2

C8 (8 of 8) : CellGroups N = 2

2023-09-16 13:15:58.891872 : Creating Coverage Files!, 0.02 mins elapsed.

2023-09-16 13:15:58.894217 : Batch Execution w/ safelapply!, 0.02 mins elapsed.

2023-09-16 13:16:48.237417 : Adding Kmer Bias to Coverage Files!, 0.842 mins elapsed.

Completed Kmer Bias Calculation

Adding Kmer Bias (1 of 16)

Adding Kmer Bias (2 of 16)

Adding Kmer Bias (3 of 16)

Adding Kmer Bias (4 of 16)

Adding Kmer Bias (5 of 16)

Adding Kmer Bias (6 of 16)

Adding Kmer Bias (7 of 16)

Adding Kmer Bias (8 of 16)

Adding Kmer Bias (9 of 16)

Adding Kmer Bias (10 of 16)

Adding Kmer Bias (11

In [13]:
getAvailableMatrices(projDLPFC1)

[1] "GeneScoreMatrix" "TileMatrix"

In [14]:
markersGS <- getMarkerFeatures(
    ArchRProj = projDLPFC1, 
    useMatrix = "GeneScoreMatrix", 
    groupBy = "Clusters",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

ArchR logging to : ArchRLogs/ArchR-getMarkerFeatures-32d9aa53f301ba-Date-2023-09-16_Time-13-17-46.913404.log
If there is an issue, please report to github with logFile!

MatrixClass = Sparse.Double.Matrix

2023-09-16 13:17:47.332666 : Matching Known Biases, 0.005 mins elapsed.

###########
2023-09-16 13:18:15.181865 : Completed Pairwise Tests, 0.47 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-getMarkerFeatures-32d9aa53f301ba-Date-2023-09-16_Time-13-17-46.913404.log



In [15]:
markerList <- getMarkers(markersGS, cutOff = "FDR <= 0.01 & Log2FC >= 1.25")
markerList$C6

DataFrame with 48 rows and 9 columns
      seqnames     start       end  strand          name     idx    Log2FC
         <Rle>   <array>   <array> <array>       <array> <array> <numeric>
7234     chr15  60016618  60014866       2 4933412E24Rik     244   2.09403
14463     chr4   6991557   6687386       2           Tox      24   1.26107
4988     chr12 106447666 106442652       2 1700121N20Rik     645   1.89738
20582     chr7 138397730 138208972       2       Tcerg1l    1986   1.62205
21070     chr8  36457548  36513013       1        Trmt9b     305   1.53670
...        ...       ...       ...     ...           ...     ...       ...
8423     chr16  56204313  56273753       1         Impg2     520   1.43218
19568     chr7  46732543  46727998       2          Saa4     972   7.00131
8989     chr17  25471590  25476594       1         Tekt4     317   1.54698
17652     chr6  50654393  50650871       2          Npvf     365   1.33966
73        chr1  20669882  20682958       1       Lincmd1      7

In [16]:
GeneScore_Matrix <- getMatrixFromProject(
  ArchRProj = projDLPFC1,
  useMatrix = "GeneScoreMatrix",
  useSeqnames = NULL,
  verbose = TRUE,
  binarize = FALSE,
  threads = getArchRThreads(),
  logFile = createLogFile("getMatrixFromProject")
)

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-32d9aa397c4ab2-Date-2023-09-16_Time-13-18-16.303782.log
If there is an issue, please report to github with logFile!

2023-09-16 13:18:39.317483 : Organizing colData, 0.384 mins elapsed.

2023-09-16 13:18:39.319947 : Organizing rowData, 0.384 mins elapsed.

2023-09-16 13:18:39.322325 : Organizing rowRanges, 0.384 mins elapsed.

2023-09-16 13:18:39.327277 : Organizing Assays (1 of 1), 0.384 mins elapsed.

2023-09-16 13:18:39.332213 : Constructing SummarizedExperiment, 0.384 mins elapsed.

2023-09-16 13:18:40.425664 : Finished Matrix Creation, 0.402 mins elapsed.



In [17]:
# rownames(peak_matrix) <- seqnames_ranges
GeneScore_Matrix

class: SummarizedExperiment 
dim: 24333 15436 
metadata(0):
assays(1): GeneScoreMatrix
rownames: NULL
rowData names(6): seqnames start ... name idx
colnames(15436): GAdBrainCortex_SNAREseq__fragments#09G_ACATCTCCGAGC
  GAdBrainCortex_SNAREseq__fragments#09I_ACTCGTTGGTCA ...
  GAdBrainCortex_SNAREseq__fragments#09I_TTATTCAACCAC
  GAdBrainCortex_SNAREseq__fragments#09J_GTGTGTAGGGGT
colData names(14): BlacklistRatio nDiFrags ... TSSEnrichment
  projColData[, colnames(projColData) %ni% colnames(colData)]

In [18]:
colData(GeneScore_Matrix)

DataFrame with 15436 rows and 14 columns
                                                        BlacklistRatio nDiFrags
                                                               <array>  <array>
GAdBrainCortex_SNAREseq__fragments#09G_ACATCTCCGAGC 0.0193031434124071     2639
GAdBrainCortex_SNAREseq__fragments#09I_ACTCGTTGGTCA 0.0293437314123323     3276
GAdBrainCortex_SNAREseq__fragments#09G_ATCTCCGAGCCC 0.0193431945697807     1349
GAdBrainCortex_SNAREseq__fragments#09A_CGGATGGCTTAT 0.0176839082959934     1126
GAdBrainCortex_SNAREseq__fragments#09L_CCCTGGGCTATG 0.0151578073089701     1402
...                                                                ...      ...
GAdBrainCortex_SNAREseq__fragments#09I_TGCACCGGAATT  0.018981018981019       95
GAdBrainCortex_SNAREseq__fragments#09K_GAGTTCTGCCCT              0.018       96
GAdBrainCortex_SNAREseq__fragments#09H_AGACATGGTCTC              0.015       45
GAdBrainCortex_SNAREseq__fragments#09I_TTATTCAACCAC              0.016       91

In [19]:
colnames(GeneScore_Matrix) <- gsub(".*#", "", colnames(GeneScore_Matrix))

In [20]:
row_Data <- rowData(GeneScore_Matrix)
rownames <- row_Data$name
rownames(GeneScore_Matrix) <- rownames

In [21]:
# 获取数据矩阵
data_matrix <- assay(GeneScore_Matrix)
# 将数据矩阵转换为稀疏矩阵
sparse_matrix <- as(data_matrix, "sparseMatrix")

In [22]:
seurat_obj <- CreateSeuratObject(counts = sparse_matrix)
obs_names <- colnames(seurat_obj)
length(obs_names)

[1] 15436

In [23]:
var_names <- rownames(seurat_obj)
length(var_names)

[1] 24333

In [24]:
rna_assay <- seurat_obj[["RNA"]]

# 查看 "RNA" assay 的值（前100个行和列）
print(rna_assay[1:100, 1:100])

100 x 100 sparse Matrix of class "dgCMatrix"


  [[ suppressing 100 column names ‘09G_ACATCTCCGAGC’, ‘09I_ACTCGTTGGTCA’, ‘09G_ATCTCCGAGCCC’ ... ]]



                                                                               
Xkr4          1.261 2.961 1.335 0.424 5.180 0.698 1.760 0.342 1.564 1.347 0.755
Rp1           0.141 1.284 .     0.127 0.403 0.584 .     .     .     .     .    
Sox17         0.311 0.552 0.188 1.042 .     0.761 .     .     .     .     .    
Mrpl15        0.175 1.929 .     0.705 .     .     .     .     0.734 .     .    
Lypla1        0.139 0.824 0.674 .     .     0.488 .     .     0.997 .     .    
Tcea1         0.188 0.228 1.001 0.588 0.500 .     .     1.014 1.481 .     0.191
Rgs20         1.161 0.622 0.777 .     .     0.751 .     0.393 0.383 1.448 0.754
Atp6v1h       0.306 0.522 0.089 .     1.146 0.588 .     0.434 .     .     0.896
Oprk1         0.048 2.661 .     1.079 .     .     .     0.841 .     0.747 .    
Npbwr1        .     1.536 .     0.362 .     .     0.376 0.313 1.133 .     .    
Rb1cc1        0.315 0.771 .     0.142 .     0.872 0.408 1.037 0.445 0.679 0.254
Alkal1        .     .     0.117 0.371 . 

In [25]:
#根据signac注释进行细胞类型注释
meta <- read.delim("/mnt/disk1/xiaojk/data/mousebrain/ATAC_celltype.txt", header = TRUE)

In [26]:
obs_cell_types <- meta$cell_type[match(obs_names, rownames(meta))]

In [27]:
meta_rownames <- rownames(meta)
cell_types_index <- match(obs_names, meta_rownames)
obs_cell_types <- obs_names[cell_types_index]
# 使用is.na()函数检查哪些元素是缺失值，然后取反以得到非缺失的布尔向量
non_missing <- !is.na(obs_cell_types)

# 使用sum()函数计算非缺失字符串的数量
non_missing_count <- sum(non_missing)
non_missing_count

[1] 6562

In [28]:
# seurat_obj_filtered$cell_type <- meta_rownames[filtered_indices]
seurat_obj$cell_type <- meta$Name[cell_types_index]
non_missing <- !is.na(seurat_obj$cell_type)

# 使用sum()函数计算非缺失字符串的数量
non_missing_count <- sum(non_missing)
# 使用 filtered_indices 子集化 Seurat 对象
seurat_obj_filtered <- seurat_obj[,non_missing]
seurat_obj_filtered

An object of class Seurat 
24333 features across 6562 samples within 1 assay 
Active assay: RNA (24333 features, 0 variable features)

In [29]:
#删除不必要的特征
seurat_obj_filtered$orig.ident <- NULL
seurat_obj_filtered$nCount_RNA <- NULL
seurat_obj_filtered$nFeature_RNA <- NULL

In [31]:
library(SeuratDisk)

SaveH5Seurat(seurat_obj_filtered, filename = "/mnt/disk1/xiaojk/data/mousebrain/mousebrain_genescore_matrix.h5Seurat")
Convert("/mnt/disk1/xiaojk/data/mousebrain/mousebrain_genescore_matrix.h5Seurat",dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Validating h5Seurat file

Adding data from RNA as X

Adding counts from RNA as raw

Transfering meta.data to obs

